# Computational Theory

## Introduction

>Things to reference later
>
> https://www.markdownguide.org/ for markdown guidance
>

## Imports

In [1]:
import numpy as np

## Technical Glossary

This section defines operations & technical terms used throughout the implementation of solutions for the problems set out in the assignment.  

### Bitwise Logical Operations
**Bitwise Operations** are operations that work on the individiual bits of a binary number. Applying the operator to each bit individually but all bits simulatenously.

**XOR (Exclusive OR)**  
Outputs 1 when the input bits are different.  

| A | B | A ⊕ B |
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   1   |
| 1 | 0 |   1   |
| 1 | 1 |   0   |

**AND**  
Outputs 1 when both input bits are 1  

| A | B | A & B |
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   0   |
| 1 | 0 |   0   |
| 1 | 1 |   1   |

**NOT**  
Output flips the bits  

| A | ¬A |
|---|----|
| 0 | 1  |
| 1 | 0  |

### Bit Manipulation Operations  

- `ROTR^n(x)`: Rotate right -  shifts bits to the right **n** times, 'fallen' bits move back to the left (start)
- `SHR^n(x)`: Logical shift right - shifts bits to the right **n** times; fills left with zeros  


### Common Terms

- **Bitwise Operation**: Performs an operation on each individual bit 
- **32-bit unsigned integer**: An integer represented by 32 individual bits in a binary sequence. "Unsigned" meaning non-negative values.
- **Diffusion**: Where the input changes are spread throughout the output  

See: [FIPS 180-4, Sections 2.2.2 and 3.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

## Problem 1 - Binary Words & Operations

### Implementing Parity

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\text{Parity}(x, y, z) = x \oplus y \oplus z$
>

  
**How it works**:  
Chains XOR operations across three 32-bit integers.  
Since XOR is a binary operation it takes 2 inputs. The expression is evaluated as $(x \oplus y) \oplus z$. The result of x XOR y is then, result XOR z.  

Each output bit is 1 if there is an odd number of input bits set to 1, otherwise 0. This is why its called "Parity" it checks the parity(odd/even count) of bits at each position.


**Example**  
X       =   0110  
Y       =   1110  
Z       =   0010  

Output  =   1010

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [2]:
def parity(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Parity: output bit is 1 if an odd number of input bits are 1.

    Parameters
    ----------
        x, y, z (int / numpy.uint32)
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The parity of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over each passed parameter
    for args in (x, y, z):
        # Use isInstance to ensure correct type (object, desired type)
        # See: https://www.w3schools.com/python/ref_func_isinstance.asp
        if not isinstance(args, (int, np.integer)):
            # Raises / Throws a TypeError with a message
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast all inputs as NumPy unsigned 32-bit integers.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Apply bitwise XOR (^) to the three inputs and return
    return x ^ y ^ z

#### Test Cases (Parity)

Checking
- Correct results for trivial values
- Correct results for non-trivial values
- Type enforcement (raise TypeError for non-integer values)

In [3]:
def test_valid_parity():
    assert parity(1, 0, 0) == np.uint32(1)      # Odd number of 1s
    assert parity(1, 1, 0) == np.uint32(0)      # Even number of 1s
    assert parity(5, 12, 6) == np.uint32(15)    # General (non-trivial) case

def test_parity_type_error():
    try:
        parity(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
test_valid_parity()
test_parity_type_error()

print("All test cases pass")

All test cases pass


### Implementing Choose (Ch)

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2 p.10) as:
>
>$\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x \land z)$
>


**How it works**:  
The ch functions uses x as a selector mask. For each bit position , if x is 1, the output takes the bit from y.  
If x is 0, the output takes the bit from z.  

This is achieved by calculating $(x \land y)$ to select bits from y where x is 1, then $(\lnot x \land z)$ to select bits from z where x is 0. Then XOR the results together.

**Example**  
X       =   1110  
Y       =   1010  
Z       =   1111  

Output  =   1011

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [ ]:
def ch(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Choose: return bits from y where x is 1, and bits from z where x is 0.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'masked' value of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over the parameters passed
    for args in (x, y, z):
        # Use isinstance + raise TypeError as before
        # See: Problem 1 -> Implementing Parity
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation and return the value
    return (x & y) ^ (~x & z)

#### Test Cases (Choose)

Checking
- Correct results for trivial values
- Correct results for non-trivial values
- Type enforcement (raise TypeError for non-integer values)

In [5]:
# Same test cases and checks as Problem 1 -> Parity
def test_valid_ch():
    assert ch(1, 0, 0) == np.uint32(0)
    assert ch(1, 1, 0) == np.uint32(1)
    assert ch(5, 12, 6) == np.uint32(6)

def test_ch_type_error():
    try:
        ch(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_valid_ch()
test_ch_type_error()

print("All test cases pass")

All test cases pass


### Implementing Majority (Maj)

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\text{maj}(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)$
>

**How it works**:  
The Maj functions implements majority voting at each bit position. For each bit, the output is 1 if atleast two input bits are 1, otherwise 0.  
This is computed by taking the AND of each pair of inputs $(x \land y)$, $(x \land z)$, and $(y \land z)$, then XOR the three results together.


**Example**  
X       =   11101110  
Y       =   10100101  
Z       =   11111001  

Output  =   11101101

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [ ]:
def maj(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Majority: output bit is 1 if two or more input bits are 1.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'voted' values of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over args, use isinstance & raise TypeError as before
    # See: Problem 1 -> Implementing Parity
    for args in (x, y, z):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation & return the value
    return (x & y) ^ (x & z) ^ (y & z)

#### Test Cases (Majority)

Checking
- Correct results for trivial values
- Correct results for non-trivial values
- Type enforcement (raise TypeError for non-integer values)

In [7]:
# Same test cases and checks as Problem 1 -> Implementing Parity
def test_maj_valid():
    assert maj(1, 0, 0) == np.uint32(0)
    assert maj(1, 1, 0) == np.uint32(1)
    assert maj(5, 12, 6) == np.uint32(4)

def test_maj_type_error():
    try:
        maj(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_maj_valid()
test_maj_type_error()

print("All test cases pass")


All test cases pass


### Implementing Big & Small Sigmas with Helper Functions

#### Helper Functions

The Sigma functions use bit rotation and shifting operations. These helper functions implement ROTR & SHR as defined in FIPS 180-4, Section 3.2 (p.9).

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, p.8-9) as:
>
>$\text{ROTR}^n(x) = (x \gg n) \lor (x \ll (w-n))$
>
>$\text{SHR}^n(x) = x \gg n$
>

- `rotr(x, n)`: Rotate right -  shifts bits to the right **n** times, 'fallen' bits move back to the left (start)
- `shr(x, n)`: Logical shift right - shifts bits to the right **n** times; fills left with zeros.

**Example**  
x = 10110011

rotr(x, 2) = 11101100  
shr(x, 4) = 00001011

See: [FIPS 180-4, Section 3.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [ ]:
def rotr(x: int, n: int) -> np.uint32:
    """
    Perform a right rotation on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int
            32-bit unsigned integer to be rotated.
        n : int
            Number of positions to rotate.

    Returns
    -------
        numpy.uint32
            The result of the right rotation as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the rotation and return the value
    return (x >> n) | (x << (32 - n))

In [ ]:
def shr(x: int, n: int) -> np.uint32:
    """
    Perform a logical right shift on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int 
            32-bit unsigned integer to be shifted.
        n : int
            Number of positions to shift.

    Returns
    -------
        numpy.uint32
            The result of the logical right shift as a 32-bit unsigned integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
        
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the logical right shift and return the value
    return x >> n

##### Test Cases (Helper Functions)

Checking
- Correct results for passed values
- Type enforcement (raise TypeError for non-integer values)

In [10]:
# Same format for tests as previous sub-problems
def test_rotr_valid():
    assert rotr(10, 1) == np.uint32(5)
    assert rotr(10, 2) == np.uint32(2147483650)
    assert rotr(16, 4) == np.uint32(1)

def test_shr_valid():
    assert shr(10, 1) == np.uint32(5)
    assert shr(10, 2) == np.uint32(2)

def test_rotr_type_error():
    try:
        rotr(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

def test_shr_type_error():
    try:
        shr(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_rotr_valid()
test_shr_valid()
test_rotr_type_error()
test_shr_type_error()

print("All test cases pass")

All test cases pass


#### Big Sigma 0 - $\Sigma_0(x)$

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\Sigma_0(x) = \text{ROTR}^{2}(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)$
>  

**How it works**:  
Rotates the input x by three different amounts (2, 13 and 22 bit positions), then XOR the three results together. This makes sure the bit mixing is rigorous.  

The rotation amounts were chosen during SHA-256's development process to ensure optimal security.

**Example**  
x = 10110011

Each bit is rotated **2**, **13** & **22** times, then XOR'd together | (2 XOR 13) XOR 22.  

See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)


In [ ]:
def big_sigma_0(x: int) -> np.uint32:
    """
    Implementing the Big Sigma 0 function used in SHA-256.

    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of ROTR(2) XOR ROTR(13) XOR ROTR(22)

    Raises
    ------
        TypeError
            If the argument is not an integer.
    """
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type
    x = np.uint32(x)

    # Perform the Big Sigma 0 operation and return the value
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))

#### Big Sigma 1 - $\Sigma_1(x)$

**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, Section 4.1.2, p.10) as:
>
>$\Sigma_1(x) = \text{ROTR}^{6}(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)$
>  

**How it works**:  
Similar to $\Sigma_0$. Rotates the input x by three different amounts (6, 11 and 25 bit positions), then XOR the three results together. This makes sure the bit mixing is rigorous.  

The rotation amounts were chosen during SHA-256's development process to ensure optimal security.
**Example**  
x = 10110011  

Each bit is rotated **6**, **11** & **25** times, then XOR'd together | (6 XOR 11) XOR 25. 


See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [ ]:
def big_sigma_1(x: int) -> np.uint32:
    """
    Implementing the Big Sigma 1 function used in SHA-256.
        
    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of the Big Sigma 1 function as a 32-bit unsigned integer.
    """
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Big Sigma 1 operation and return the value
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

##### Test Cases (Big Sigma(s))

In [13]:
def test_bsigma_0_valid():
    assert big_sigma_0(1)  == np.uint32(rotr(1, 2) ^ rotr(1, 13) ^ rotr(1, 22))
    assert big_sigma_0(5)  == np.uint32(rotr(5, 2) ^ rotr(5, 13) ^ rotr(5, 22))
    assert big_sigma_0(10) == np.uint32(rotr(10, 2) ^ rotr(10, 13) ^ rotr(10, 22))

def test_bsigma_1_valid():
    assert big_sigma_1(1)  == np.uint32(rotr(1, 6) ^ rotr(1, 11) ^ rotr(1, 25))
    assert big_sigma_1(5)  == np.uint32(rotr(5, 6) ^ rotr(5, 11) ^ rotr(5, 25))
    assert big_sigma_1(10) == np.uint32(rotr(10, 6) ^ rotr(10, 11) ^ rotr(10, 25))


def test_bsigma_type_error():
    try:
        big_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        big_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_bsigma_0_valid()
test_bsigma_1_valid()
test_bsigma_type_error()

print("All test cases pass")

All test cases pass


#### Small Sigma 0 & 1
- sigma 0 -> XOR Bitwise, 2 rotations (7, 18) & Bitwise shift right (3) - SHA-256 Reference required here
- sigma 1 -> XOR Bitwise, 2 rotations (17, 19) & Bitwise shift right (10) - SHA-256 Reference required here

In [14]:
def small_sigma_0(x):
    """
    Implementing the Small Sigma 0 function used in SHA-256.

    Args:
        x (int): 32-bit unsigned integer.
    
    Returns:
        int: Result of the Small Sigma 0 function as a 32-bit unsigned integer.
    """

    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))

In [15]:
def small_sigma_1(x):
    """
    Implementing the Small Sigma 1 function used in SHA-256.

    Args:
        x (int): 32-bit unsigned integer.
    
    Returns:
        int: Result of the Small Sigma 1 function as a 32-bit unsigned integer.
    """

    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

##### Test Cases (Small Sigma(s))

In [16]:
def test_ssigma_0_valid():
    assert small_sigma_0(1)  == np.uint32(rotr(1, 7) ^ rotr(1, 18) ^ shr(1, 3))
    assert small_sigma_0(5)  == np.uint32(rotr(5, 7) ^ rotr(5, 18) ^ shr(5, 3))
    assert small_sigma_0(10) == np.uint32(rotr(10, 7) ^ rotr(10, 18) ^ shr(10, 3))

def test_ssigma_type_error():
    try:
        small_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        small_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_ssigma_0_valid()
test_ssigma_type_error()

print("All test cases pass")

All test cases pass


## Problem 2 - Fractional Parts of Cube Roots

## Problem 3 - Padding

## Problem 4 - Hashes

## Problem 5 - Passwords

## Conclusion

## End